In [2]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.8/272.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 29.7 MB/s eta 0:00:00
  Created

# Try Apache Beam - Python

In this notebook, we set up your development environment and work through a simple example using the [DirectRunner](https://beam.apache.org/documentation/runners/direct/). You can explore other runners with the [Beam Capatibility Matrix](https://beam.apache.org/documentation/runners/capability-matrix/).

To navigate through different sections, use the table of contents. From **View**  drop-down list, select **Table of contents**.

To run a code cell, you can click the **Run cell** button at the top left of the cell, or by select it and press **`Shift+Enter`**. Try modifying a code cell and re-running it to see what happens.

To learn more about Colab, see [Welcome to Colaboratory!](https://colab.sandbox.google.com/notebooks/welcome.ipynb).

# Setup

First, you need to set up your environment, which includes installing `apache-beam` and downloading a text file from Cloud Storage to your local file system. We are using this file to test your pipeline.

In [1]:
import apache_beam as beam
import re

# Input text file
input_file = "alice_large.txt"

print("Dataset ready:", input_file)

Dataset ready: alice_large.txt


# Word count with comments

Below is mostly the same code as above, but with comments explaining every line in more detail.

In [10]:
import apache_beam as beam
import re

# DoFn class used to extract words from each line (different method from original code)
class ExtractWordsFn(beam.DoFn):
    def process(self, element):
        # Convert line to lowercase and extract only alphabetic words
        words = re.findall(r"[A-Za-z]+", element.upper())
        for w in words:
            yield w  # emit each word

output_path = "part"

with beam.Pipeline() as pipeline:

    word_counts = (
        pipeline
        | "ReadFile" >> beam.io.ReadFromText(input_file)
          # Read the text line by line

        | "ExtractWords" >> beam.ParDo(ExtractWordsFn())
          # NEW LOGIC: Use DoFn to extract words (instead of FlatMap)

        | "CountWords" >> beam.combiners.Count.PerElement()
          # NEW METHOD: Count how many times each word appears

        | "FormatOutput" >> beam.Map(lambda wc: str((wc[0].upper(), wc[1])))
          # Format as "word: count"

    )

    # Write output to txt file
    word_counts | "WriteOutput" >> beam.io.WriteToText(
        output_path,
        file_name_suffix=".txt"
    )

print("Output file created at:", output_path + "-00000-of-00001.txt")
print("Showing first 50 lines:\n")

# Print first few lines from generated file
with open(output_path + "-00000-of-00001.txt", "r") as f:
    for line in f.readlines()[:50]:
        print(line.strip())

Output file created at: part-00000-of-00001.txt
Showing first 50 lines:

('CHAPTER', 200)
('I', 200)
('DOWN', 400)
('THE', 1800)
('RABBIT', 600)
('HOLE', 400)
('ALICE', 400)
('WAS', 600)
('BEGINNING', 200)
('TO', 400)
('GET', 200)
('VERY', 400)
('TIRED', 200)
('OF', 1000)
('SITTING', 200)
('BY', 400)
('HER', 1000)
('SISTER', 400)
('ON', 400)
('BANK', 200)
('AND', 1000)
('HAVING', 200)
('NOTHING', 200)
('DO', 200)
('ONCE', 200)
('OR', 600)
('TWICE', 200)
('SHE', 600)
('HAD', 400)
('PEEPED', 200)
('INTO', 200)
('BOOK', 400)
('READING', 200)
('BUT', 200)
('IT', 400)
('NO', 200)
('PICTURES', 400)
('CONVERSATIONS', 400)
('IN', 400)
('WHAT', 200)
('IS', 200)
('USE', 200)
('A', 800)
('THOUGHT', 200)
('WITHOUT', 200)
('SO', 200)
('CONSIDERING', 200)
('OWN', 200)
('MIND', 200)
('AS', 400)
